In [1]:
import pandas as pd
import numpy as np


In [14]:
data = pd.read_csv('LabTests.csv',encoding='ISO-8859-1')
data

,LOINC_CD,Description,value,UNITS,MAX_RANGE,MIN_RANGE,GENDER
0,2028-9,"CARBON DIOXIDE, TOTAL",27,MMOL/L,29,20,F
1,13458-5,VLDL CHOLESTEROL CAL,13,MG/DL,40,5,F
2,2345-7,GLUCOSE,96,MG/DL,99,65,F
3,88293-6,EGFR IF AFRICN AM,61,ML/MIN/1.73,0,0,F
4,1751-7,ALBUMIN,3.5,G/DL,4.7,3.5,F
...,...,...,...,...,...,...,...
189400,2885-2,"PROTEIN, TOTAL",6.4,G/DL,8.5,6,F
189401,2823-3,POTASSIUM,5.8,MMOL/L,5.2,3.5,F
189402,2571-8,TRIGLYCERIDES,63,MG/DL,149,0,F
189403,6768-6,ALKALINE PHOSPHATASE,81,IU/L,117,39,F


**1. Ranges when it’s null and units are missing; especially when we have only 1 patient that has done the test**

*Handle Null Ranges and Missing Units:*


*   **For cases where the range is null and units are missing, we can consider using the following approach:**



1.  If we have multiple patients with the same test, we can calculate the range for each test based on their values and use the median value of the range as the max and min range.
2.   If we only have one patient with the test, we can consider using the normal range for that test from the literature or a clinical guideline (Best solution : but I cannot obtain the normal range for a test from a reference table or clinical guideline ) one alternative approach you can consider is to use the interquartile range (IQR) of the values for that test as an estimate of the normal range



        
*   **For cases where the units are missing, we can search for the correct unit for that test using the LOINC code and add it to the dataset**



In [15]:
# Group the dataframe by LOINC code and Description
grouped = data.groupby(['LOINC_CD', 'Description'])
grouped.head()

,LOINC_CD,Description,value,UNITS,MAX_RANGE,MIN_RANGE,GENDER
0,2028-9,"CARBON DIOXIDE, TOTAL",27,MMOL/L,29,20,F
1,13458-5,VLDL CHOLESTEROL CAL,13,MG/DL,40,5,F
2,2345-7,GLUCOSE,96,MG/DL,99,65,F
3,88293-6,EGFR IF AFRICN AM,61,ML/MIN/1.73,0,0,F
4,1751-7,ALBUMIN,3.5,G/DL,4.7,3.5,F
...,...,...,...,...,...,...,...
189040,11579-0,THYROTROPIN,3.52,UIU/ML,NaN,NaN,F
189043,38518-7,GRANULOCYTES.IMMATURE/100 LEUKOCYTES,0,%,NaN,NaN,F
189047,2501-5,IRON BINDING CAPACITY.UNSATURATED,285,UG/DL,NaN,NaN,F
189216,3051-0,TRIIODOTHYRONINE.FREE,2.6,PG/ML,NaN,NaN,F


some statistics about the null ranges and missing units problem before solving it

In [16]:
# Count the number of null ranges and missing units for each group
null_range_counts = grouped['MAX_RANGE'].apply(lambda x: x.isnull().all()).value_counts()
missing_units_counts = grouped['UNITS'].apply(lambda x: x.isnull().all()).value_counts()

# Print the counts
print('Number of tests with null ranges: ')
print(null_range_counts)
print('\nNumber of tests with missing units: ')
print(missing_units_counts)

Number of tests with null ranges: 
True     4627
False    4520
Name: MAX_RANGE, dtype: int64

Number of tests with missing units: 
False    5844
True     3303
Name: UNITS, dtype: int64



*   **Number of tests with null ranges:** This output tells you that out of all the tests in the data, there are 4627 tests that have null (missing) values in their MAX_RANGE column, and there are 4520 tests that have non-null values in their MAX_RANGE column. This is because the groupby operation groups the data by the LOINC_CD and Description columns, and the apply operation is used to check whether all values in the MAX_RANGE column for each group are null or not.

*   **Number of tests with missing units:** This output tells you that out of all the tests in the data, there are 5844 tests that have non-null values in their UNITS column, and there are 3303 tests that have null (missing) values in their UNITS column. This is because the groupby operation groups the data by the LOINC_CD and Description columns, and the apply operation is used to check whether all values in the UNITS column for each group are null or not.

*True and False values in the output correspond to boolean values indicating whether all values in the MAX_RANGE or UNITS column for each group are null or not. The value_counts() function is used to count the number of groups that have True and False values.*




In [23]:
# simple solution 1 but you need to to obtain this information from a reference source, 
#such as the LOINC database. Additionally, you could fill in missing units using similar methods, 
#such as cross-referencing with a reference source or consulting with a lab technician or physician who is familiar with the test

# Replace null ranges with appropriate values based on LOINC code
data.loc[data['MAX_RANGE'].isnull(), 'MAX_RANGE'] = data['LOINC_CD'].map({
    '2028-9': 30,
    '1234-5': 200,
    # -------> Add additional LOINC codes and range values as needed <----------
    # -------> Add additional LOINC codes and range values as needed <----------
    # -------> Add additional LOINC codes and range values as needed <----------

})

# Fill in missing units based on LOINC code
data.loc[data['UNITS'].isnull(), 'UNITS'] = data['LOINC_CD'].map({
    '2028-9': 'mmol/L',
    '1234-5': 'g/dL',
    # -------> Add additional LOINC codes and range values as needed <----------
    # -------> Add additional LOINC codes and range values as needed <----------
    # -------> Add additional LOINC codes and range values as needed <----------

})


# Convert the 'value' column to a numeric data type
data['value'] = pd.to_numeric(data['value'], errors='coerce')

# Identify and remove outliers based on mean and standard deviation
mean = data['value'].mean()
std_dev = data['value'].std()
data = data[(data['value'] >= mean - 3 * std_dev) & (data['value'] <= mean + 3 * std_dev)]

# Save the cleaned and corrected lab test data to a new CSV file
data.to_csv('cleaned_lab_test_data.csv', index=False)

In [24]:
cleand_data = pd.read_csv('cleaned_lab_test_data.csv')
cleand_data.head()

,LOINC_CD,Description,value,UNITS,MAX_RANGE,MIN_RANGE,GENDER
0,2028-9,"CARBON DIOXIDE, TOTAL",27.0,MMOL/L,29,20,F
1,13458-5,VLDL CHOLESTEROL CAL,13.0,MG/DL,40,5,F
2,2345-7,GLUCOSE,96.0,MG/DL,99,65,F
3,88293-6,EGFR IF AFRICN AM,61.0,ML/MIN/1.73,0,0,F
4,1751-7,ALBUMIN,3.5,G/DL,4.7,3.5,F


Second Solution : get_normal_range complete implementation to obtain the normal range for each test based on the LOINC_CD column

In [28]:
# second solution
def get_iqr(test_values):
    q1, q3 = np.percentile(test_values, [25, 75])
    return q3 - q1

def get_normal_range(loinc_cd):
    # Replace this placeholder code with your implementation to obtain the normal range for each test based on the LOINC_CD column
    # Example: assume that we have a dictionary of normal ranges for each test
    normal_ranges = {'2085-9': [3.6, 5.4], '2339-0': [70, 100], '4548-4': [3.8, 5.0]}
    if loinc_cd in normal_ranges:
        return normal_ranges[loinc_cd]
    else:
        return None

# Handle Null Ranges and Missing Units

# Replace missing units with the most common unit for that test
data['UNITS'] = data.groupby('LOINC_CD')['UNITS'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else 'unknown'))

# Get normal ranges for each test
normal_ranges = data.groupby('LOINC_CD').apply(lambda x: pd.Series({'normal_range': get_normal_range(x['LOINC_CD'].iloc[0])}))

# Handle Null Ranges
# Calculate the range for each test based on their values and use the median value of the range as the max and min range
null_range = data['MAX_RANGE'].isnull()
null_unit = data['UNITS'].isnull()
grouped_data = data[~null_range & ~null_unit].groupby('LOINC_CD')
for name, group in grouped_data:
    if len(group) > 1:
        max_val = group['value'].max()
        min_val = group['value'].min()
        max_range = max_val - min_val
        data.loc[(null_range | null_unit) & (data['LOINC_CD'] == name), 'MAX_RANGE'] = max_range
        data.loc[(null_range | null_unit) & (data['LOINC_CD'] == name), 'MIN_RANGE'] = max_range
    elif len(group) == 1:
        iqr = get_iqr(group['value'])
        data.loc[(null_range | null_unit) & (data['LOINC_CD'] == name), 'MAX_RANGE'] = group['value'].iloc[0] + iqr
        data.loc[(null_range | null_unit) & (data['LOINC_CD'] == name), 'MIN_RANGE'] = group['value'].iloc[0] - iqr

# Save the cleaned data to a new csv file
data.to_csv('cleaned_lab_test_data.csv', index=False)

<ipython-input-28-db6060e3d8d7>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['UNITS'] = data.groupby('LOINC_CD')['UNITS'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else 'unknown'))


In [32]:
cleaned_data = pd.read_csv('cleaned_lab_test_data.csv')
cleaned_data

,LOINC_CD,Description,value,UNITS,MAX_RANGE,MIN_RANGE,GENDER
0,2028-9,"CARBON DIOXIDE, TOTAL",27.0,MMOL/L,29,20,F
1,13458-5,VLDL CHOLESTEROL CAL,13.0,MG/DL,40,5,F
2,2345-7,GLUCOSE,96.0,MG/DL,99,65,F
3,88293-6,EGFR IF AFRICN AM,61.0,ML/MIN/1.73,0,0,F
4,1751-7,ALBUMIN,3.5,G/DL,4.7,3.5,F
...,...,...,...,...,...,...,...
165157,2885-2,"PROTEIN, TOTAL",6.4,G/DL,8.5,6,F
165158,2823-3,POTASSIUM,5.8,MMOL/L,5.2,3.5,F
165159,2571-8,TRIGLYCERIDES,63.0,MG/DL,149,0,F
165160,6768-6,ALKALINE PHOSPHATASE,81.0,IU/L,117,39,F


2. Values are out of range and therefore min and max that we decide on are incorrect





In [34]:
# Define a function to calculate the normal range for a given LOINC_CD
def get_normal_range(loinc_cd):
    # Replace this placeholder code with your implementation to obtain the normal range for each test based on the LOINC_CD column
    # Example: assume that we have a dictionary of normal ranges for each test
    normal_ranges = {'2085-9': [3.6, 5.4], '2339-0': [70, 100], '4548-4': [3.8, 5.0]}
    if loinc_cd in normal_ranges:
        return normal_ranges[loinc_cd]
    else:
        return None

# Filter data to include only rows within the normal range
cleand_data = cleaned_data.copy()
cleand_data['normal_range'] = cleand_data['LOINC_CD'].apply(get_normal_range)
cleand_data = cleand_data[cleand_data['normal_range'].notnull()]
cleand_data = cleand_data[cleand_data['value'].between(cleand_data['normal_range'].apply(lambda x: x[0]), cleand_data['normal_range'].apply(lambda x: x[1]))]

# Calculate min and max range based on filtered data
min_range = cleand_data.groupby('LOINC_CD')['value'].min()
max_range = cleand_data.groupby('LOINC_CD')['value'].max()

# Replace values outside of range with NaN
cleand_data.loc[~cleand_data['value'].between(min_range.loc[cleand_data['LOINC_CD']].values, max_range.loc[cleand_data['LOINC_CD']].values), 'value'] = np.nan

In [35]:
cleand_data

,LOINC_CD,Description,value,UNITS,MAX_RANGE,MIN_RANGE,GENDER,normal_range
2162,4548-4,HEMOGLOBIN A1C,5.0,%,5.6,4.8,F,"[3.8, 5.0]"
2381,2339-0,POCGLU,96.0,MILLIGRAMS/DECILIT,100,65,M,"[70, 100]"
2414,2339-0,POCGLU,99.0,MILLIGRAMS/DECILIT,100,65,M,"[70, 100]"
2425,2339-0,POCGLU,86.0,MILLIGRAMS/DECILIT,100,65,M,"[70, 100]"
2426,2339-0,POCGLU,87.0,MILLIGRAMS/DECILIT,100,65,M,"[70, 100]"
...,...,...,...,...,...,...,...,...
155731,4548-4,HEMOGLOBIN A1C/HEMOGLOBIN.TOTAL IN BLOOD,4.7,%,10.4,10.4,F,"[3.8, 5.0]"
157941,2339-0,GLUCOSE,74.0,MG/DL,449.0,449.0,F,"[70, 100]"
158922,2085-9,"HDL CHOLESTEROL, SERUM",3.7,MG/DL,128.0,128.0,M,"[3.6, 5.4]"
160585,2339-0,ARTERIAL FULL PANEL,99.0,MG/DL,99,74,M,"[70, 100]"
